In [91]:
import requests
import time 

ws_id = "13d64313-5498-42f3-bf8b-9b2b83e1a4d4"
lakehouse_id= "3d0191f6-e627-47c7-a289-5b201bc45ca9"

api_base_url_mist='https://msitapi.fabric.microsoft.com/v1/'
api_base_url_daily ='https://dailyapi.fabric.microsoft.com/v1/'
api_base_url_dxt = 'https://dxtapi.fabric.microsoft.com/v1/'

api_url = api_base_url_daily

with open('token.txt', 'r') as file:
    access_token = file.read().strip()

headers = {"Authorization": "Bearer " + access_token}

def printResponse(response):
   print(f"API call return with status code: {response.status_code}")
   print(response.json())
   print(response.text)


In [ ]:
# call the list workspace API to check if the token is valid

get_ws_url= api_url+"workspaces/ff84063b-d6c5-4684-b427-c17c881b6110"
get_ws_response = requests.get(get_ws_url, headers=headers)
print(get_ws_response.json())

In [ ]:
# create a new ws and assign the capacity 
# create_ws_url= api_url+"workspaces"

# payload_data = {
#     "displayName": "WS4SparkAPIDemo"
# }
# create_ws_response = requests.post(create_ws_url, headers=headers,json=payload_data)
# print(create_ws_response.json())
# new_ws_id = create_ws_response.json()['id']

new_ws_id = 'ff84063b-d6c5-4684-b427-c17c881b6110'

# assignCapacity_url = api_url +"workspaces/" +new_ws_id+ "assignToCapacity"
# assignCapacity_payload ={
#     "capacityId": "0f084df7-c13d-451b-af5f-ed0c466403b2"
# }

In [ ]:
# list the ws settings related with Spark
get_spark_ws_settings =  api_url+"workspaces/"+new_ws_id+"/spark/settings/"
get_spark_settings_response = requests.get(get_spark_ws_settings, headers=headers)
printResponse(get_spark_settings_response)

In [ ]:
# let's create custom pool 
create_pool_url = api_url+"workspaces/"+new_ws_id+"/spark/pools"
wild_cat = 'wildcat2'
create_pool_payload = {
    "name": wild_cat,
    "nodeFamily": "MemoryOptimized",
    "nodeSize": "Small",
    "autoScale": {
        "enabled": 'true',
        "minNodeCount": 1,
        "maxNodeCount": 2
    },
    "dynamicExecutorAllocation": {
        "enabled": 'true',
        "minExecutors": 1,
        "maxExecutors": 1
    }
}
create_pool_response = requests.post(create_pool_url, headers=headers,json=create_pool_payload)
printResponse(create_pool_response)
new_pool_id = create_pool_response.json()['id']

In [ ]:
# try to get all the pools 
get_pool_url = api_url+"workspaces/"+new_ws_id+"/spark/pools"
get_pool_url = requests.get(get_pool_url, headers=headers)
printResponse(get_pool_url)

In [ ]:
# update the ws spark settings by setting the default pool to the new pool and set the default runtime version

starter_pool = "Starter Pool"
another_pool = "wildcat"
update_spark_setting_url = api_url+"workspaces/"+new_ws_id+"/spark/settings/"
update_spark_setting_payload={
    "pool": {
        "customizeComputeEnabled": 'true',
        "defaultPool": {
            "name": starter_pool,
            "type": "Workspace"
        }
    },
    "environment": {
        "runtimeVersion": "1.2"
    }
}
update_spark_setting_response = requests.patch(update_spark_setting_url, headers=headers, json=update_spark_setting_payload)
printResponse(update_spark_setting_response)

In [ ]:
# now let create a new environment artifact and set it as the default env of the workspace

newenv = "starterEnv3"

create_env_url = api_url + "/workspaces/"+new_ws_id+"/items"
create_env_payload = {
    "displayName": newenv,
    "type": 'Environment'
}
create_env_response = requests.post(create_env_url, headers=headers, json=create_env_payload)
printResponse(create_env_response)

# update this env with certain settings

env_id = create_env_response.json()['id']
#env_id = '3c52ddf8-df2e-45c0-9fdb-55f7b9fbcbb7'
update_env_url = api_url + "/workspaces/"+new_ws_id+"/environments/"+env_id+"/staging/sparkcompute"
update_env_payload = {
    "instancePool": {
    "name": "Starter Pool",
    "type": "Workspace"
  },
  "driverCores": 4,
  "driverMemory": "28g",
  "executorCores": 4,
  "executorMemory": "28g",
  "dynamicExecutorAllocation": {
    "enabled": 'true',
    "minExecutors": 1,
    "maxExecutors": 2
  },
  "sparkProperties": {
    "spark.setting1": "setting2222",
    "spark.setting2": "settings3"
  },
  "runtimeVersion": "1.1"
}
update_env_response = requests.patch(update_env_url, headers=headers, json=update_env_payload)
printResponse(update_env_response)

# set this env as the default env of the workspace

set_default_env_url = api_url+"workspaces/"+new_ws_id+"/spark/settings/"
set_default_env_payload={
    "environment": {
        "name": "starterEnv3"
    }
}
set_default_env_response = requests.patch(set_default_env_url, headers=headers, json=set_default_env_payload)
printResponse(set_default_env_response)

In [ ]:
# create a new Lakehouse with the name as LHAPIDemo within this workspace

newlakehouseName = "LHAPIDemo"

create_lakehouse = api_url + "/workspaces/"+new_ws_id+"/items"
create_lakehouse_payload = {
    "displayName": newlakehouseName,
    "type": 'Lakehouse'
}
create_lakehouse_response = requests.post(create_lakehouse, headers=headers, json=create_lakehouse_payload)
printResponse(create_lakehouse_response)
lakehouse_id = create_lakehouse_response.json()['id']

# Assume under the File section of this lakehouse, we have a file named "yellow_tripdata_2022_01.csv"
# Now we can call the Load to Table API to create a delta table from this csv file, a new table named "yellowtrip" will be created under the Lakehouse

#lakehouse_id ='0a48b3d9-b277-4555-a0cd-f87804a4e821'
tablename = "yellowtrip"
load_to_table_url = api_url + "/workspaces/"+new_ws_id+"/lakehouses/"+lakehouse_id+"/tables/"+tablename+"/load "
load_to_table_payload = {
    "relativePath": "Files/yellow_tripdata_2022_01.csv",
    "pathType": "File",
    "mode": "overwrite",
    "formatOptions": {
        "header": 'true', 
        "delimiter": ",", 
        "format": "CSV" 
    }
}

load_to_table_response = requests.post(load_to_table_url, headers=headers, json=load_to_table_payload)

print(load_to_table_response.headers['Location'])
operation_id = load_to_table_response.headers['Location'].split('/')[-1]
print(operation_id)

# Now we can call the get operation to check the status of the load operation

# get_load_status_url = api_url + "/workspaces/"+new_ws_id+"/lakehouses/"+lakehouse_id+"/operations/"+operation_id
# get_load_status_response = requests.get(get_load_status_url, headers=headers)
# print(get_load_status_response.json())


In [ ]:
# now let's create a new notebook with one code cell as : 
# df = spark.sql("SELECT * FROM LHAPIDemo.`yellow_tripdata` LIMIT 1000")
# display(df) 

new_notebook_name = "NotebookAPIDemo"
new_notebook_content_in_base64 = "eyJjZWxscyI6W3siY2VsbF90eXBlIjoiY29kZSIsInNvdXJjZSI6WyJkZiA9IHNwYXJrLnNxbChcIlNFTEVDVCAqIEZST00gTEhBUElEZW1vLmB5ZWxsb3dfdHJpcGRhdGFgIExJTUlUIDEwMDBcIilcbiIsImRpc3BsYXkoZGYpIl0sIm91dHB1dHMiOltdLCJleGVjdXRpb25fY291bnQiOm51bGwsIm1ldGFkYXRhIjp7fSwiaWQiOiI2YWM3YTAwZC1iMjhiLTRjZjctYmQ4OC05ZWMzY2E5NjhkYzcifV0sIm1ldGFkYXRhIjp7Imxhbmd1YWdlX2luZm8iOnsibmFtZSI6InB5dGhvbiJ9LCJrZXJuZWxfaW5mbyI6eyJuYW1lIjoic3luYXBzZV9weXNwYXJrIn0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiJ9LCJ3aWRnZXRzIjp7fSwibnRlcmFjdCI6eyJ2ZXJzaW9uIjoibnRlcmFjdC1mcm9udC1lbmRAMS4wLjAifSwic3BhcmtfY29tcHV0ZSI6eyJjb21wdXRlX2lkIjoiL3RyaWRlbnQvZGVmYXVsdCJ9LCJkZXBlbmRlbmNpZXMiOnsibGFrZWhvdXNlIjp7ImRlZmF1bHRfbGFrZWhvdXNlIjoiMGE0OGIzZDktYjI3Ny00NTU1LWEwY2QtZjg3ODA0YTRlODIxIiwiZGVmYXVsdF9sYWtlaG91c2VfbmFtZSI6IkxIQVBJRGVtbyIsImRlZmF1bHRfbGFrZWhvdXNlX3dvcmtzcGFjZV9pZCI6ImZmODQwNjNiLWQ2YzUtNDY4NC1iNDI3LWMxN2M4ODFiNjExMCJ9fX0sIm5iZm9ybWF0Ijo0LCJuYmZvcm1hdF9taW5vciI6NX0="

create_notebook_url = api_url + "/workspaces/"+new_ws_id+"/items"
create_notebook_payload = {
    "displayName": new_notebook_name,
    "type": 'Notebook',
    "definition" : {
        "format": "ipynb",
        "parts": [
            {
                "path": "notebook-content.ipynb",
                "payload": new_notebook_content_in_base64,
                "payloadType": "InlineBase64"
            }
        ]
    }
}
create_notebook_response = requests.post(create_notebook_url, headers=headers, json=create_notebook_payload)
printResponse(create_notebook_response)

#Now let's trigger the execution of this notebook 

notebook_id = create_notebook_response.json()['id']
#notebook_id = '1326f424-ccb5-4a53-9734-8f4c15bcb748'    
trigger_notebook_url = api_url + "/workspaces/"+new_ws_id+"/items/"+notebook_id+"/jobs/instances?jobType=RunNotebook"
trigger_notebook_payload = {
    "executionData": {
        "parameters": {
            "para1": {
                "value": "new path",
                "type": "string"
            }
        }
    }
}
trigger_notebook_response = requests.post(trigger_notebook_url, headers=headers, json=trigger_notebook_payload)
print(trigger_notebook_response.headers['Location'])
job_id = trigger_notebook_response.headers['Location'].split('/')[-1]
print(job_id)

# Now let's check the status of this job
# job_id = '2c02a9fd-4df7-4b8a-8c0a-47f94c78c6e3'
# notebook_id = '1326f424-ccb5-4a53-9734-8f4c15bcb748'
get_job_status_url = api_url + "/workspaces/"+new_ws_id+"/items/"+notebook_id+"/jobs/instances/"+job_id
get_job_status_response = requests.get(get_job_status_url, headers=headers)
print(get_job_status_response.json())




In [ ]:
# Now let's create a new Livy session and execute a statement to query the data in the delta table

lakehouse_id ='f715aa5d-493f-401b-8f8c-62d9bcd19f57'
ws_id = '833c95bb-47e2-4592-b5ff-39ccb854cc39'
livy_base_url = api_base_url_mist + "/workspaces/"+ws_id+"/lakehouses/"+lakehouse_id +"/livyApi/versions/2023-12-01/"

create_livy_session = livy_base_url + "sessions"
payload_data = {}
create_session = requests.post(create_livy_session, headers=headers,json=payload_data)
session_id = create_session.json()['id']

print('The request to create Livy session is submitted:' + str(create_session.json()))

get_livy_session =  livy_base_url+"sessions/" + session_id
get_session_response = requests.get(get_livy_session, headers=headers)
#print(get_session_response.json())

while get_session_response.json()["state"] != "idle":
    # Sleep for 5 seconds before making the next request
    time.sleep(5)
    #print('the statement code is submitted and running  : ' + str(execute_statement_response.json()))

    # Make the next request
    get_session_response = requests.get(get_livy_session, headers=headers)


execute_statement = livy_base_url + "sessions/" + session_id +"/statements"
payload_data =    {
      "code": "spark.sql(\"SELECT * FROM ContosoOutdoors.Transactions where TotalPrice < 0\").show()",
      "kind": "spark"
    }
execute_statement_response = requests.post(execute_statement, headers=headers,json=payload_data)
#print('/////////////////////////')
#print(get_session_response.json())
print('the statement code is submitted as: ' + str(execute_statement_response.json()))

statement_id = str(execute_statement_response.json()['id'])
get_statement = livy_base_url +"sessions/"+ session_id + "/statements/" + statement_id
get_statement_response = requests.get(get_statement, headers=headers)

while get_statement_response.json()["state"] != "available":
    # Sleep for 5 seconds before making the next request
    time.sleep(5)
    print('the statement code is submitted and running  : ' + str(execute_statement_response.json()))

    # Make the next request
    get_statement_response = requests.get(get_statement, headers=headers)


rst = get_statement_response.json()
 
# Assuming result is your returned data
result_data = rst['output']['data']['text/plain']
print(result_data)

In [94]:
# call get batch API
lakehouse_id ='f715aa5d-493f-401b-8f8c-62d9bcd19f57'
ws_id = '833c95bb-47e2-4592-b5ff-39ccb854cc39'
livy_base_url = api_base_url_mist + "/workspaces/"+ws_id+"/lakehouses/"+lakehouse_id +"/livyApi/versions/2023-12-01/"
get_livy_get_batch = livy_base_url+"batches"
get_batch_response = requests.get(get_livy_get_batch, headers=headers)
print(get_batch_response.text)

{"items":[{"id":"6a8b0395-06df-43d4-bbb1-64dcd269025c","name":"livybatchdemo_withLHAPIDemo2","workspaceId":"833c95bb-47e2-4592-b5ff-39ccb854cc39","submitterId":"a7d2bd4a-be4e-4d7d-8787-a07cdcb4e95b","submitterName":"qixwang@microsoft.com","artifactId":"f715aa5d-493f-401b-8f8c-62d9bcd19f57","result":"Uncertain","submittedAt":"2024-03-21T11:30:51.0293977+00:00","startedAt":"2024-03-21T11:30:57.209397+00:00","tags":{"ArtifactJobType":"LivyBatch","JobInvokeType":"Livy API","maxAllowedCoresForJob":"80","artifactName":"FabricLivyDemo","operationName":"Batch Livy Run","appliedThrottling":"{\"ThrottlingType\":0,\"Value\":0.0}","isBillable":"True","tenantId":"72f988bf-86f1-41af-91ab-2d7cd011db47","capacityId":"483c2de8-fb78-4b2a-bc72-3218d2764314","workspaceId":"833c95bb-47e2-4592-b5ff-39ccb854cc39","artifactId":"f715aa5d-493f-401b-8f8c-62d9bcd19f57","artifactKind":"Lakehouse","consumerIdentity":"","fabricRuntimeVersion":"1.2"},"schedulerState":"Scheduled","pluginState":"Monitoring","livyState"

In [93]:
# Now let's submit a spark job via the livy batch API, this job will run a .py script set by the path in the payload.
# The spark job will write the result to delta table in the lakehouse created as LHAPIDemo2
 
lakehouse_id ='f715aa5d-493f-401b-8f8c-62d9bcd19f57'
ws_id = '833c95bb-47e2-4592-b5ff-39ccb854cc39'
livy_base_url = api_base_url_mist + "/workspaces/"+ws_id+"/lakehouses/"+lakehouse_id +"/livyApi/versions/2023-12-01/"


newlakehouseName = "LHAPIDemo2"

create_lakehouse = api_base_url_mist + "/workspaces/"+ws_id+"/items"
create_lakehouse_payload = {
    "displayName": newlakehouseName,
    "type": 'Lakehouse'
}
create_lakehouse_response = requests.post(create_lakehouse, headers=headers, json=create_lakehouse_payload)
printResponse(create_lakehouse_response)


run_batch_job = livy_base_url+"batches"
payload_data = {
    "name":"livybatchdemo_with"+ newlakehouseName,
    "file":"abfss://ContosoOutdoor@msit-onelake.dfs.fabric.microsoft.com/ContosoOutdoors.Lakehouse/Files/livybatch/livybatchdemo.py",
    "conf": {
        "spark.targetLakehouse": newlakehouseName

    }
}
get_batch_response = requests.post(run_batch_job, headers=headers,json=payload_data)

print("The Livy batch job submitted successful")
print(get_batch_response.json())


API call return with status code: 201
{'id': '6eadf6aa-97eb-4153-9478-2e07c0ca0f6b', 'type': 'Lakehouse', 'displayName': 'LHAPIDemo2', 'description': '', 'workspaceId': '833c95bb-47e2-4592-b5ff-39ccb854cc39'}
{"id":"6eadf6aa-97eb-4153-9478-2e07c0ca0f6b","type":"Lakehouse","displayName":"LHAPIDemo2","description":"","workspaceId":"833c95bb-47e2-4592-b5ff-39ccb854cc39"}
The Livy batch job submitted successful
{'id': '6a8b0395-06df-43d4-bbb1-64dcd269025c', 'artifactId': 'f715aa5d-493f-401b-8f8c-62d9bcd19f57'}
